In [3]:
import pandas as pd
patients = pd.read_csv("Patient Matching Data.csv")

In [4]:
patients.head()

,GroupID,PatientID,Patient Acct #,First Name,MI,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,...,Current State,Current Zip Code,Previous First Name,Previous MI,Previous Last Name,Previous Street 1,Previous Street 2,Previous City,Previous State,Previous Zip Code
0,1,1,247028705-7,Sutton,J,Power,9/20/1945,Male,1858 Sullivan Parkway,NaN,...,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apt 9,Mount Vernon,New York,10557.0
1,1,2,NaN,Suttin,James,Power,9/21/1945,Male,1859 Sullivan Parkway,#2,...,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apartment # 9,Mount Vernon,New York,10557.0
2,1,3,247028705-7,Sutton,J,Power,9/20/1945,Male,1858 Sullivan Parkway,NaN,...,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,NaN,Sutton,NaN,Power,9/20/1954,Male,1858 Sullivan Parkway,NaN,...,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,NaN,SUTTON,NaN,POWER,9/20/1954,Male,1858 SULLIVAN PKWAY,APT 2,...,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
cleanPatients = patients.drop(["GroupID", "PatientID", "MI"], axis =1)
cleanPatients.head()

,Patient Acct #,First Name,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,Current City,Current State,Current Zip Code,Previous First Name,Previous MI,Previous Last Name,Previous Street 1,Previous Street 2,Previous City,Previous State,Previous Zip Code
0,247028705-7,Sutton,Power,9/20/1945,Male,1858 Sullivan Parkway,NaN,Fresno,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apt 9,Mount Vernon,New York,10557.0
1,NaN,Suttin,Power,9/21/1945,Male,1859 Sullivan Parkway,#2,Fresno,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apartment # 9,Mount Vernon,New York,10557.0
2,247028705-7,Sutton,Power,9/20/1945,Male,1858 Sullivan Parkway,NaN,Fresno,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Sutton,Power,9/20/1954,Male,1858 Sullivan Parkway,NaN,Fresno,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,SUTTON,POWER,9/20/1954,Male,1858 SULLIVAN PKWAY,APT 2,FRESNO,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
def validateAcc(acc):
    if len(str(acc)) != 11:
        return (float('NaN'))
    else:
        if acc[9] != "-":
            return (float('NaN'))
        else:
            return acc

cleanPatients["Patient Acct #"] = cleanPatients["Patient Acct #"].apply(validateAcc)


In [21]:
cleanPatients.head(30)

,Patient Acct #,First Name,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,Current City,Current State,Current Zip Code,Previous First Name,Previous MI,Previous Last Name,Previous Street 1,Previous Street 2,Previous City,Previous State,Previous Zip Code
0,247028705-7,Sutton,Power,9/20/1945,Male,1858 Sullivan Parkway,NaN,Fresno,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apt 9,Mount Vernon,New York,10557.0
1,NaN,Suttin,Power,9/21/1945,Male,1859 Sullivan Parkway,#2,Fresno,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apartment # 9,Mount Vernon,New York,10557.0
2,247028705-7,Sutton,Power,9/20/1945,Male,1858 Sullivan Parkway,NaN,Fresno,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Sutton,Power,9/20/1954,Male,1858 Sullivan Parkway,NaN,Fresno,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,SUTTON,POWER,9/20/1954,Male,1858 SULLIVAN PKWAY,APT 2,FRESNO,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Sutton,Powers,12/15/2000,Female,1619 Maple Lane,NaN,Fresno,California,93726.0,Sutton,NaN,Lore,NaN,NaN,NaN,NaN,NaN
6,NaN,Sutton,Powers,12/15/2000,Female,1619 Maple Lane,NaN,Fresno,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,757393348-7,Mikel,Trayton,9/20/2000,Male,715 Jana Point,NaN,Apache Junction,Arizona,85219.0,NaN,NaN,NaN,81089 Kenwood Park,NaN,Chattanooga,Tennessee,37416.0
8,757393348-7,Mikel,Trayton,9/20/2000,Male,715 Jana Point,NaN,Apache Junction,Arizona,85219.0,NaN,NaN,NaN,81089 Kenwood Park,NaN,Chattanooga,Tennessee,37416.0
9,757393348-7,Michael,Trayton,9/20/2000,Male,715 Jana Point,NaN,Apache Junction,AZ,85219.0,NaN,NaN,NaN,81089 Kenwood Park,7,Chattanooga,Tennessee,37416.0


In [22]:
tokenized_name = cleanPatients['First Name'].apply(lambda x: x.split())

tokenized_name.head()

0    [Sutton]
1    [Suttin]
2    [Sutton]
3    [Sutton]
4    [SUTTON]
Name: First Name, dtype: object

In [23]:
from nltk import PorterStemmer

ps = PorterStemmer()

tokenized_name = tokenized_name.apply(lambda x: [ps.stem(i) for i in x])

tokenized_name.head()

0    [sutton]
1    [suttin]
2    [sutton]
3    [sutton]
4    [sutton]
Name: First Name, dtype: object

In [24]:
tokenized_name.head(40)

0      [sutton]
1      [suttin]
2      [sutton]
3      [sutton]
4      [sutton]
5      [sutton]
6      [sutton]
7       [mikel]
8       [mikel]
9     [michael]
10       [mike]
11      [mikel]
12      [mikel]
13      [dalia]
14      [dalia]
15     [myrvyn]
16     [mervin]
17     [myrvin]
18     [mervyn]
19     [mervin]
20      [cammi]
21        [cam]
22    [cameron]
23     [elenor]
24     [elenor]
25    [elenora]
26      [janet]
27      [janet]
28      [darbi]
29      [darbi]
30    [maynard]
31    [maynard]
32    [maynard]
33    [maynard]
34       [dant]
35       [dant]
36       [rena]
37    [annetta]
38        [ann]
39     [annett]
Name: First Name, dtype: object

In [27]:
 def lowerstr(string):
        return(string.lower())
    
cleanPatients["First Name"] = cleanPatients['First Name'].apply(lowerstr)
cleanPatients.head()

,Patient Acct #,First Name,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,Current City,Current State,Current Zip Code,Previous First Name,Previous MI,Previous Last Name,Previous Street 1,Previous Street 2,Previous City,Previous State,Previous Zip Code
0,247028705-7,sutton,Power,9/20/1945,Male,1858 Sullivan Parkway,NaN,Fresno,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apt 9,Mount Vernon,New York,10557.0
1,NaN,suttin,Power,9/21/1945,Male,1859 Sullivan Parkway,#2,Fresno,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apartment # 9,Mount Vernon,New York,10557.0
2,247028705-7,sutton,Power,9/20/1945,Male,1858 Sullivan Parkway,NaN,Fresno,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,sutton,Power,9/20/1954,Male,1858 Sullivan Parkway,NaN,Fresno,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,sutton,POWER,9/20/1954,Male,1858 SULLIVAN PKWAY,APT 2,FRESNO,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
cleanPatients["First Name"].value_countscounts()

cheston        7
pasquale       7
worthington    7
katharina      6
kirbee         6
              ..
pascale        1
anne-marie     1
rena           1
mervyn         1
ray            1
Name: First Name, Length: 99, dtype: int64